This is the tutorial for the evaluation of CPHF alchemical derivatives using PySCF (version 1.7.6)  

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf
pyscf.__version__

'1.7.6'

## Fractional charge molecules

In [2]:
from FcMole import FcM, FcM_like

In [3]:
mol_NN=gto.M(atom= "N 0 0 0; N 0 0 2.1",unit="Bohr", basis="sto-3g") 

The FcM_like function creates a copy of a givem PySCF molecule object with modified nuclear cherges 

In [4]:
fmol=FcM_like(mol_NN,fcs=[.2,-.1])

Only nuclear charges are modified, the number of electrons stays the same

In [5]:
fmol.atom_charges(),fmol.nelec

(array([7.2, 6.9]), (7, 7))

The same result can be achieved using the FcM constructor

In [6]:
fmol1=FcM(fcs=[.2,-.1],atom= "N 0 0 0; N 0 0 2.1",unit="Bohr", basis="sto-3g")

The number of electron matches the atom string

In [7]:
fmol1.atom_charges(),fmol1.nelec

(array([7.2, 6.9]), (7, 7))

If only some atoms have modified nuclear charges is possible to pass as a fcs parameter a double list [[idxs],[fcs]]

In [8]:
fmol2=FcM(fcs=[[0,1],[.2,-.1]],atom= "N 0 0 0; N 0 0 2.1",unit="Bohr", basis="sto-3g")

In [9]:
fmol2.atom_charges(),fmol2.nelec

(array([7.2, 6.9]), (7, 7))

For scf calculations using fractional charges molecules the initial density matrix guess needs to be evaluated from the eigenfunction of the monoelectronic Hamiltonian (there is no single atom density "SAD" matrix guess for atoms with fractional charge)

In [26]:
mf=scf.RHF(fmol)
mf1=scf.RHF(fmol1)
mf2=scf.RHF(fmol2)

In [27]:
mf.scf(dm0=mf.init_guess_by_1e())
mf1.scf(dm0=mf1.init_guess_by_1e())
mf2.scf(dm0=mf2.init_guess_by_1e())

converged SCF energy = -108.621905368439
converged SCF energy = -108.621905368439
converged SCF energy = -108.621905368439


-108.62190536843865

## Using the Alchemical Perturbator (AP) class
We created an Alchemical perturbator object to access easily all the CPHF methods for alchemical derivative and alchemical forces derivatives, this allow us to save the derivatives and also to exploit the symmetry of the system

In [12]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP

The alchemical perturbator is instantiated from a converged RHF object, and for some given perturbation sites.

In [13]:
mf_nn=scf.RHF(mol_NN)
mf_nn.scf()
ap_nn=AP(mf_nn,sites=[0,1])

converged SCF energy = -107.49885049543


Alchemical gradient $ \partial E/\partial Z_i $, hessian $\partial^2E/\partial Z_i\partial Z_j$ and cubic hessian $\partial^3E/\partial Z_i\partial Z_j\partial Z_k$can be obtained from their buid functions 


In [14]:
#build the alchemical gradient dE/dZ_i
ap_nn.build_gradient()

array([-17.96252176, -17.96252176])

In [15]:
#build the alchemical hessian d**2E/dZ_i/dZ_j
ap_nn.build_hessian()

array([[-0.56261427,  0.95186321],
       [ 0.95186321, -0.56261427]])

In [16]:
ap_nn.build_cubic_hessian()

array([[[0.05028813, 0.00297524],
        [0.00297524, 0.00297524]],

       [[0.00297524, 0.00297524],
        [0.00297524, 0.05028813]]])

Are saved inside the class and can be accessed in a later moment

In [17]:
ap_nn.gradient,ap_nn.hessian,ap_nn.cubic_hessian

(array([-17.96252176, -17.96252176]),
 array([[-0.56261427,  0.95186321],
        [ 0.95186321, -0.56261427]]),
 array([[[0.05028813, 0.00297524],
         [0.00297524, 0.00297524]],
 
        [[0.00297524, 0.00297524],
         [0.00297524, 0.05028813]]]))

Alchemical perturbations for isolelectronic transmutations can be calculated from the derivatives up to order 3

In [18]:
ap_nn.APDFT1(np.asarray([-1,0])) # to CN-

-89.53632873812936

In [19]:
ap_nn.APDFT2(np.asarray([-1,1])) # to CO

-109.01332797130455

In [20]:
ap_nn.APDFT3(np.asarray([0,1]))  # to NO+

-125.73429803367387

In [46]:
ap_nn.APDFT3(np.asarray([1,1])) # to OO++

-143.01490712587176

## Alchemical forces
Alchemical forces are calculated resusing the density matrix derivatives already evaluated. The function af(i) gives the alchemical force of the atom $i$ : $\partial \mathbf{g}/ \partial Z_i$ [2]


In [22]:
ap_nn.af(0),ap_nn.af(1)

(array([[ 1.14240350e-16, -2.57554516e-16,  1.62250798e-01],
        [-1.14240350e-16,  2.57554516e-16, -1.62250798e-01]]),
 array([[ 5.84952962e-17, -9.81316581e-17,  1.62250798e-01],
        [-5.84952962e-17,  9.81316581e-17, -1.62250798e-01]]))

## Basis set effects 
The class also include methods to calculate the energy of the target molecules, with its basis set and with the basis set of the reference,

In [23]:
ap_nn.target_energy([-1,1]),ap_nn.target_energy_ref_bs([-1,1])

converged SCF energy = -111.221914650137
converged SCF energy = -108.996149557208


(-111.22191465013725, -108.99614955720807)

The APDFT3 predictions approximate the energy of the molecule with the reference basis set 

In [24]:
ap_nn.APDFT3([-1,1])

-109.01332797130455

But we can correct it using the single atom basis set correction [1]

In [25]:
ap_nn.APDFT3([-1,1])+ap_nn.ap_bsec([-1,1])

-111.47291530309323

## References 

[1] Giorgio Domenichini, Guido Falk von Rudorff, and O. Anatole von Lilienfeld : "Effects of perturbation order and basis set on alchemical predictions", J. Chem. Phys. 153, 144118 (2020)

[2] Giorgio Domenichini, and O. Anatole von Lilienfeld: "Alchemical predictions of relaxed geometries throughout chemical space", under review (2021) 